# this should be in backend
testing here for convenience

In [1]:
import os
import json
import pandas as pd

import torch
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util

import mysql.connector
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
from sqlalchemy import text as sqtext
load_dotenv(find_dotenv())

from svs import SVS
from queryOpenAI import run_engine

/Users/isaac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

In [3]:
# preloading slow things
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')
f = open('../models/mimic-iii/qqdict.json')
qqdict = json.load(f)
corpus_embedding = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

client = OpenAI(api_key=os.getenv("OAI_KEY"))

/Users/isaac/Library/Python/3.9/lib/python/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
conn = engine.connect()

2024-02-11 04:57:16,897 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 04:57:16,898 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 04:57:17,070 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 04:57:17,071 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 04:57:17,143 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 04:57:17,143 INFO sqlalchemy.engine.Engine [raw sql] {}


In [12]:
question = "what is the ingesting method of lansoprazole oral suspension?"

In [13]:
def backend(question):
    qqlist = [qqdict[str(i)] for i in SVS(question, k=15, corpus=corpus_embedding, model=model)]

    relevant_qtext = "# Q: \t" + "\n# Q: \t".join(["\n# A: \t".join(rel) for rel in qqlist])
    test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

    result = run_engine(question, test_prompt, relevant_qtext)

    print(result)
    print()
    return result
    
backend(question)

# Q: 	what are the methods of consumption of insulin human nph?
# A: 	select distinct prescriptions.route from prescriptions where prescriptions.drug = 'insulin human nph'
# Q: 	tell me the intake method for readi-cat 2 (barium sulfate 2% suspension)?
# A: 	select distinct prescriptions.route from prescriptions where prescriptions.drug = 'readi-cat 2 (barium sulfate 2% suspension)'
# Q: 	just how many metoprolol tartrate prescriptions were there since 4 years ago?
# A: 	select count(*) from prescriptions where prescriptions.drug = 'metoprolol tartrate' and datetime(prescriptions.startdate) >= datetime(current_time,'-4 year')
# Q: 	what are the ways to consume sterile water?
# A: 	select distinct prescriptions.route from prescriptions where prescriptions.drug = 'sterile water'
# Q: 	tell me the intake method for sirolimus?
# A: 	select distinct prescriptions.route from prescriptions where prescriptions.drug = 'sirolimus'
# Q: 	what are the ways to consume guaifenesin?
# A: 	select disti

"SELECT DISTINCT PRESCRIPTIONS.ROUTE \nFROM PRESCRIPTIONS \nWHERE PRESCRIPTIONS.DRUG = 'LANSOPRAZOLE ORAL SUSPENSION';"

In [20]:
test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

In [5]:
print(question)
print()
print("SELECT COUNT(*)>0 \nFROM PRESCRIPTIONS \nWHERE PRESCRIPTIONS.HADM_ID IN \n( \n    SELECT ICUSTAYS.HADM_ID \n    FROM ICUSTAYS \n    WHERE ICUSTAYS.SUBJECT_ID = 22973 \n    ORDER BY ICUSTAYS.INTIME DESC \n    LIMIT 1 \n) \nAND PRESCRIPTIONS.DRUG = 'INSULIN';")

Was patient 22973 prescribed insulin the last time they were at the ICU?

SELECT COUNT(*)>0 
FROM PRESCRIPTIONS 
WHERE PRESCRIPTIONS.HADM_ID IN 
( 
    SELECT ICUSTAYS.HADM_ID 
    FROM ICUSTAYS 
    WHERE ICUSTAYS.SUBJECT_ID = 22973 
    ORDER BY ICUSTAYS.INTIME DESC 
    LIMIT 1 
) 
AND PRESCRIPTIONS.DRUG = 'INSULIN';


In [6]:
text = '''SELECT DISTINCT PRESCRIPTIONS.ROUTE FROM PRESCRIPTIONS WHERE PRESCRIPTIONS.DRUG = 'LANSOPRAZOLE ORAL SUSPENSION';'''

NameError: name 'sqlalchemy' is not defined

In [12]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
connection = engine.connect()
result = connection.execute(text("SHOW TABLES;"))
result

2024-02-11 04:56:57,580 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 04:56:57,580 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 04:56:57,743 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 04:56:57,743 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 04:56:57,815 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 04:56:57,816 INFO sqlalchemy.engine.Engine [raw sql] {}


TypeError: 'str' object is not callable